In [1]:
import puz
import json

In [6]:
class Variable():

    ACROSS = "across"
    DOWN = "down"

    def __init__(self, i, j, direction, length):
        """Create a new variable with starting point, direction, and length."""
        self.i = i
        self.j = j
        self.direction = direction
        self.length = length
        self.cells = []
        for k in range(self.length):
            self.cells.append(
                (self.i + (k if self.direction == Variable.DOWN else 0),
                 self.j + (k if self.direction == Variable.ACROSS else 0))
            )

    def __hash__(self):
        return hash((self.i, self.j, self.direction, self.length))

    def __eq__(self, other):
        return (
            (self.i == other.i) and
            (self.j == other.j) and
            (self.direction == other.direction) and
            (self.length == other.length)
        )

    def __str__(self):
        return f"({self.i}, {self.j}) {self.direction} : {self.length}"

    def __repr__(self):
        direction = repr(self.direction)
        return f"Variable({self.i}, {self.j}, {direction}, {self.length})"

class Crossword():
    def __init__(self, grid):
        self.structure = []

        self.height = len(grid) # the number of rows in the grid
        self.width = len(grid[0]) # the number of columns in the grid
        for i in range(len(grid)):
            row = []
            for j in range(len(grid[0])):
                if grid[i][j] == '':
                    row.append(False)
                else:
                    row.append(True)
            self.structure.append(row)



        # Determine variable set
        self.variables = set()

        for i in range(self.height):
            for j in range(self.width):

                # Vertical words
                starts_word = (
                    self.structure[i][j]
                    and (i == 0 or not self.structure[i - 1][j])
                )
                if starts_word:
                    length = 1
                    for k in range(i + 1, self.height):
                        if self.structure[k][j]:
                            length += 1
                        else:
                            break
                    if length > 1:
                        self.variables.add(Variable(
                            i=i, j=j,
                            direction=Variable.DOWN,
                            length=length
                        ))

                # Horizontal words
                starts_word = (
                    self.structure[i][j]
                    and (j == 0 or not self.structure[i][j - 1])
                )
                if starts_word:
                    length = 1
                    for k in range(j + 1, self.width):
                        if self.structure[i][k]:
                            length += 1
                        else:
                            break
                    if length > 1:
                        self.variables.add(Variable(
                            i=i, j=j,
                            direction=Variable.ACROSS,
                            length=length
                        ))

In [2]:
def puz_to_json(fname):
    """ Converts a puzzle in .puz format to .json format
    """
    p = puz.read(fname)
    numbering = p.clue_numbering()

    grid = [[None for _ in range(p.width)] for _ in range(p.height)]
    for row_idx in range(p.height):
        cell = row_idx * p.width
        row_solution = p.solution[cell:cell + p.width]
        for col_index, item in enumerate(row_solution):
            if p.solution[cell + col_index:cell + col_index + 1] == '.':
                grid[row_idx][col_index] = 'BLACK'
            else:
                grid[row_idx][col_index] = ["", row_solution[col_index: col_index + 1]]

    across_clues = {}
    for clue in numbering.across:
        answer = ''.join(p.solution[clue['cell'] + i] for i in range(clue['len']))
        across_clues[str(clue['num'])] = [clue['clue'] + ' ', ' ' + answer]
        grid[int(clue['cell'] / p.width)][clue['cell'] % p.width][0] = str(clue['num'])

    down_clues = {}
    for clue in numbering.down:
        answer = ''.join(p.solution[clue['cell'] + i * numbering.width] for i in range(clue['len']))
        down_clues[str(clue['num'])] = [clue['clue'] + ' ', ' ' + answer]
        grid[int(clue['cell'] / p.width)][clue['cell'] % p.width][0] = str(clue['num'])


    mydict = {'metadata': {'date': None, 'rows': p.height, 'cols': p.width}, 'clues': {'across': across_clues, 'down': down_clues}, 'grid': grid}
    return mydict

In [4]:
json_data = puz_to_json("./crossword_01-05-2024.puz")

In [39]:
output_grid = [  ['D', 'I', 'L', 'E', 'M', 'M', 'A', '', 'L', 'O', 'C', 'U', 'M'], 
                 ['I', '', 'O', '', 'I', '', 'E', '', 'I', '', 'E', '', 'O'], 
                 ['C', 'R', 'O', 'W', 'D', '', 'R', 'I', 'N', 'G', 'L', 'E', 'T'], 
                 ['K', '', 'S', '', 'W', '', 'A', '', 'K', '', 'S', '', 'E'], 
                 ['E', 'V', 'E', 'R', 'E', 'S', 'T', '', 'S', 'K', 'I', 'L', 'L'], 
                 ['N', '', '', '', 'E', '', 'E', '', '', '', 'U', '', ''], 
                 ['S', 'N', 'A', 'C', 'K', 'S', '', 'C', 'O', 'R', 'S', 'E', 'T'], 
                 ['', '', 'M', '', '', '', 'M', '', 'L', '', '', '', 'H'], 
                 ['C', 'O', 'U', 'C', 'H', '', 'E', 'L', 'D', 'E', 'R', 'L', 'Y'], 
                 ['O', '', 'S', '', 'A', '', 'A', '', 'H', '', 'I', '', 'R'], 
                 ['B', 'E', 'I', 'J', 'I', 'N', 'G', '', 'A', 'U', 'D', 'I', 'O'], 
                 ['R', '', 'N', '', 'R', '', 'R', '', 'N', '', 'E', '', 'I'], 
                 ['A', 'N', 'G', 'R', 'Y', '', 'E', 'N', 'D', 'U', 'R', 'E', 'D']]

In [40]:
def evaluate(puzzle, solved_grid):
    no_rows, no_cols = puzzle['metadata']['rows'], json_data['metadata']['cols']
    
    grid_structure = []
    grid_solution = []

    for row_e in json_data['grid']:
        row = []
        solution_row = []
        for col_e in row_e:
            if not isinstance(col_e, list):
                solution_row.append('')
                row.append('')
            else:
                solution_row.append(col_e[1])
                row.append('A')
        grid_structure.append(row)
        grid_solution.append(solution_row)
        
    crossword = Crossword(grid_structure)
    
    letters_correct = 0
    words_correct = 0

    total_letters = 0
    total_words = 0
    for slot in crossword.variables:
        total_words += 1
        ith_row, jth_col = slot.i, slot.j
        ans_len = slot.length
        ans_direction = slot.direction
        total_letters += ans_len
        temp_letter_count = 0

        if ans_direction == 'across':
            for k in range(jth_col, jth_col + ans_len):
                if grid_solution[ith_row][k] == output_grid[ith_row][k]:
                    temp_letter_count += 1

        elif ans_direction == 'down':
            for k in range(ith_row, ith_row + ans_len):
                if grid_solution[k][jth_col] == output_grid[k][jth_col]:
                    temp_letter_count += 1

        if temp_letter_count == ans_len:
            words_correct += 1

        letters_correct += temp_letter_count

    return letters_correct / total_letters, words_correct / total_words

In [41]:
evaluate(json_data, output_grid)

(1.0, 1.0)